In [1]:
import numpy as np 
import pandas as pd
import os
import torch
from PIL import Image
import tensorflow as tf
import datasets
from torch.utils.data import Dataset as torch_ds
import torchvision.transforms as tfm
import re
from kaggle_datasets import KaggleDatasets
import tensorflow_addons as tfa

2023-02-17 19:02:54.109678: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-17 19:02:54.109806: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
CONFIG = {
    'seed': 42,
    'buffer_size': 2048,
    'device': 'tpu',
    'root_directory': KaggleDatasets().get_gcs_path('tpu-getting-started'),
    'lr':1e-5,
    'batch_size_train': 4,
    'batch_size_test': 20,
    'parallel_setting': tf.data.experimental.AUTOTUNE,
    'image_size': [192, 192],
    'epochs':30,
    'optimizer': 'adam',
    'loss_function' : 'categorical_crossentropy'
}

CONFIG['images'] = ''.join([CONFIG['root_directory'], '/tfrecords-jpeg-512x512'])
CONFIG['testing_filenames'] = tf.io.gfile.glob(CONFIG['images'] + '/test/*.tfrec')

'''
CONFIG['training_filenames'] = tf.io.gfile.glob(CONFIG['images_512x512'] + '/train/*.tfrec')
CONFIG['validation_filenames'] = tf.io.gfile.glob(CONFIG['images_512x512'] + '/val/*.tfrec')
CONFIG['testing_filenames'] = tf.io.gfile.glob(CONFIG['images_512x512'] + '/test/*.tfrec')
'''
CONFIG['classes'] = [   
    'pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
    'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
    'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
    'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
    'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
    'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
    'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
    'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
    'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
    'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
    'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose'
]  

2023-02-17 19:03:01.314377: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


In [3]:
#print(tf.io.gfile.glob(CONFIG['images_512x512'] + '/train/*.tfrec'))
CONFIG['training_filenames'] = []
CONFIG['validation_filenames'] = []
for fold in os.listdir('../input/tpu-getting-started')[1:]:
    CONFIG['training_filenames'].extend(tf.io.gfile.glob(CONFIG['root_directory'] + '/' + fold + '/train/*.tfrec'))
    CONFIG['validation_filenames'].extend(tf.io.gfile.glob(CONFIG['root_directory'] + '/' + fold + '/val/*.tfrec'))

2023-02-17 19:03:01.405653: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-02-17 19:03:01.483530: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-02-17 19:03:01.558152: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-02-17 19:03:01.638057: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.
2023-02-17 19:03:01.710868: I tensorflow/core/platform/cloud/google_auth_provider.cc:180] Attempting an empty bearer token since no token was retrieved from files, and GCE metadata check was skipped.


In [4]:
len(CONFIG['training_filenames']), len(CONFIG['validation_filenames']), len(CONFIG['testing_filenames']), 

(64, 64, 16)

In [5]:
def set_seed(seed=42):
    np.random.seed(seed)
    tf.random.set_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark
    os.environ['PYTHONHASHSEED'] = str(seed)
set_seed(CONFIG['seed'])

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

2023-02-17 19:03:02.042961: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-17 19:03:02.045117: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-17 19:03:02.045170: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-17 19:03:02.045219: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ac32ab32b3a6): /proc/driver/nvidia/version does not exist
2023-02-17 19:03:02.049083: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Running on TPU  grpc://10.0.0.2:8470


2023-02-17 19:03:02.057155: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-17 19:03:02.088349: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-17 19:03:02.088407: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30020}
2023-02-17 19:03:02.106870: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-17 19:03:02.106934: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30020}
2023-02-17 19:03:02.108876: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:411] Started server with target: grpc://localhost:30020


REPLICAS:  8


In [6]:
class FlowersDataset(torch_ds):
    def data_augment(self, image, label):
        # Thanks to the dataset.prefetch(AUTO)
        # statement in the next function (below), this happens essentially
        # for free on TPU. Data pipeline code is executed on the "CPU"
        # part of the TPU while the TPU itself is computing gradients.
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_saturation(image, 0, 2)
        return image, label  
    
    def decode_image(self,image_data):
        image = tf.image.decode_jpeg(image_data, channels=3)
        image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
        image = tf.image.resize(image, CONFIG['image_size'])
        image = tf.reshape(image, [*CONFIG['image_size'], 3]) # explicit size needed for TPU
        return image

    def read_labeled_tfrecord(self, example):
        LABELED_TFREC_FORMAT = {
            "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
            "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
        }
        example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
        image = self.decode_image(example['image'])
        label = tf.cast(example['class'], tf.int32)
        return [image,  label] #{'image': image, 'label': label, 'idx' :idx} # returns a dataset of (image, label) pairs

    def read_unlabeled_tfrecord(self, example):    
        LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        }
        example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
        image = self.decode_image(example['image'])
        image = tf.cast(image, tf.float32)
        idx = tf.cast(example['id'], tf.string) 
        return [image, idx] 
    
    def load_dataset(self, filenames, labeled=True, ordered=False):
        # Read from TFRecords. For optimal performance, reading from multiple files at once and
        # disregarding data order. Order does not matter since we will be shuffling the data anyway.

        ignore_order = tf.data.Options()
        if not ordered:
            ignore_order.experimental_deterministic = False # disable order, increase speed

        dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=CONFIG['parallel_setting']) # automatically interleaves reads from multiple files
        
        
        dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
        dataset = dataset.map(self.read_labeled_tfrecord if labeled else self.read_unlabeled_tfrecord, num_parallel_calls=CONFIG['parallel_setting'])
        # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
        return dataset
    
    def __init__(self, train_filenames,
            valid_filenames, test_filenames, path):
        self.train_filenames = train_filenames
        self.valid_filenames = valid_filenames
        self.test_filenames = test_filenames
        self.path = path

    def save_dataset(self, path):
        tf.data.experimental.save(self.ds, path)
        
    def load_processed_dataset(self, path):
        return tf.data.experimental.load(path, reader_func=self.custom_reader_func)
        
    def create_datasets(self):
        self.train_ds = self.load_dataset(self.train_filenames)
        self.train_ds = self.train_ds.map(self.data_augment, num_parallel_calls=CONFIG['parallel_setting'])
        self.val_ds = self.load_dataset(self.valid_filenames)
        self.test_ds = self.load_dataset(self.test_filenames, labeled=False,ordered=False)
    def apply_z_score_normalization(self): 
        dataset_filter = self.train_ds.map(lambda x1, x2: x1).as_numpy_iterator()
        mean = 0.
        std = 0.

        cnt = 0
        for np_array in dataset_filter:
            mean += np_array.mean()
            std += np_array.std()
            cnt+=1
        mean = mean/cnt
        std = std/cnt 
        
        self.train_ds = self.train_ds.map(lambda x1, x2,: ((x1 - mean)/std, x2))
        self.val_ds = self.val_ds.map(lambda x1, x2: ((x1 - mean)/std, x2))
        self.test_ds = self.test_ds.map(lambda x1, x2: ((x1 - mean)/std, x2))
        
    def create_batch_of_datasets(self):
        self.train_ds = self.train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=len(CONFIG['classes']))))
        self.val_ds = self.val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=len(CONFIG['classes']))))
        
        self.train_ds = self.train_ds.shuffle(buffer_size=CONFIG['buffer_size'], seed=CONFIG['seed'])
        self.train_ds = self.train_ds.batch(CONFIG['batch_size_train'])
        self.train_ds = self.train_ds.prefetch(CONFIG['parallel_setting']) 
        self.val_ds = self.val_ds.batch(CONFIG['batch_size_train'])
        self.val_ds = self.val_ds.prefetch(CONFIG['parallel_setting'])
        self.test_ds = self.test_ds.batch(CONFIG['batch_size_test'])
        self.test_ds = self.test_ds.prefetch(CONFIG['parallel_setting'])

In [7]:
dataset = FlowersDataset(CONFIG['training_filenames'],
                         CONFIG['validation_filenames'],
                         CONFIG['testing_filenames'], CONFIG['root_directory'])
dataset.create_datasets()
dataset.apply_z_score_normalization()
dataset.create_batch_of_datasets()
train_ds, val_ds, test_ds = dataset.train_ds, dataset.val_ds, dataset.test_ds

2023-02-17 19:05:08.921124: W ./tensorflow/core/distributed_runtime/eager/destroy_tensor_handle_node.h:57] Ignoring an error encountered when deleting remote tensors handles: Invalid argument: Unable to find the relevant tensor remote_handle: Op ID: 102056, Output num: 0
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1676660708.917813922","description":"Error received from peer ipv4:10.0.0.2:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Unable to find the relevant tensor remote_handle: Op ID: 102056, Output num: 0","grpc_status":3}


In [8]:
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)

with strategy.scope():
    pretrained_model = tf.keras.applications.VGG16(
        weights='imagenet',
        include_top=False ,
        input_shape=[*CONFIG['image_size'], 3]
    )
    
    pretrained_model.trainable = False
    
    model = tf.keras.Sequential([
        # To a base pretrained on ImageNet to extract features from images...
        pretrained_model,
        # ... attach a new head to act as a classifier.
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CONFIG['classes']), activation='softmax')
    ])
    
    f1_metric = tfa.metrics.F1Score(num_classes=len(CONFIG['classes']), threshold=0.5, average='micro')
    
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

class FlowerClassifier:
    def __init__(self, model, metric, lr_callback, strategy, CONFIG):
        self.model = model
        self.metric = metric
        self.lr_callback = lr_callback
        self.strategy = strategy
        
        self.model.compile(
            optimizer=CONFIG['optimizer'],
            loss = CONFIG['loss_function'],
            metrics=[self.metric],
        )

        self.model.summary()
        
    def fit(self, train_ds, val_ds):         
        history = self.model.fit(
            train_ds,
            validation_data=val_ds,
            epochs=CONFIG['epochs'],
        )
        
flwrcls = FlowerClassifier(model, f1_metric,lr_callback, strategy, CONFIG)

58892288/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 6, 6, 512)         14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 104)               53352     
Total params: 14,768,040
Trainable params: 53,352
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
flwrcls.fit(train_ds, val_ds)